In [12]:
from openai import OpenAI
import numpy as np
import faiss
from tqdm import tqdm
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
api_key = "sk-07b0bbfd24bf4cb391cad5da8da05f6f"
base_url = "https://dashscope.aliyuncs.com/compatible-mode/v1"
model_name = "text-embedding-v3"
BATCH_SIZE  = 32
TOP_K       = 3  

# from langchain_community.embeddings import DashScopeEmbeddings
# embeddings = DashScopeEmbeddings(
#     model="text-embedding-v2",
#     dashscope_api_key="sk-0b8d48b85f1742829ef3032133375d3e")
# llm = ChatOpenAI(base_url='https://dashscope.aliyuncs.com/compatible-mode/v1',
#                  api_key="sk-0b8d48b85f1742829ef3032133375d3e",
#                  model="qwen2.5-72b-instruct", temperature=0.7)
# client_openai = OpenAI(
#     api_key= "sk-07b0bbfd24bf4cb391cad5da8da05f6f",
#     base_url= "https://dashscope.aliyuncs.com/compatible-mode/v1" 
# )

## Dual‑Encoder 检索

In [14]:
import os

import faiss
import numpy as np
from openai import OpenAI
from tqdm import tqdm

'''
        和以下的区别，这里可以做成LLM生成答案，支持支持 IndexIVFFlat 带训练的倒排索引
        quantizer = faiss.IndexFlatIP(dim)
        index = faiss.IndexIVFFlat(quantizer, dim, nlist)
        if not index.is_trained:index.train(vectors)
        index.add(vectors)
'''
''' 只做检索,使用 IndexFlatIP 或归一化后做余弦相似度检索,可扩展为多文档结构
 faiss.normalize_L2(doc_vectors)
dim = doc_vectors.shape[1]
index = faiss.IndexFlatIP(dim)           # 内积，归一化后≈余弦
index.add(doc_vectors)
 '''
api_key = "sk-07b0bbfd24bf4cb391cad5da8da05f6f"
base_url = "https://dashscope.aliyuncs.com/compatible-mode/v1"
model_name = "text-embedding-v3"
BATCH_SIZE  = 32
TOP_K       = 3
embedding_client = OpenAI(
        api_key=api_key,
        base_url=base_url
    )
def embed_texts(texts):
    resp = embedding_client.embeddings.create(
            model=model_name,
            input=batch,
            #dimensions=dimensions,
            encoding_format="float"
        )
    # OpenAI v1 返回顺序与输入保持一致
    return np.array([d.embedding for d in resp.data], dtype="float32")

documents = [
    "RAG 是 Retrieval‑Augmented Generation 的缩写。",
    "Dual‑Encoder 采用查询和文档两个塔独立编码。",
    "OpenAI 提供 text‑embedding‑3 系列模型。",
    "在图谱检索中常用图数据库 Neo4j。",
]
doc_vectors = []
for i in tqdm(range(0, len(documents), BATCH_SIZE)):
    batch = documents[i:i+BATCH_SIZE]
    doc_vectors.append(embed_texts(batch))

doc_vectors = np.vstack(doc_vectors)

# ── 2.3 可选：余弦检索 → 先做 L2 归一化
faiss.normalize_L2(doc_vectors)

# ── 2.4 建立 FAISS 索引（余弦 = 归一化后 L2）
dim = doc_vectors.shape[1]
index = faiss.IndexFlatIP(dim)           # 内积，归一化后≈余弦
index.add(doc_vectors)
def search(query, k=TOP_K):
    # 3.1 编码查询
    q_vec = embed_texts([query])
    faiss.normalize_L2(q_vec)

    # 3.2 ANN 检索
    scores, indices = index.search(q_vec, k)
    return [(documents[i], float(scores[0][j])) for j, i in enumerate(indices[0])]

# 测试
query = "如何用双塔做语义检索？"
for rank, (text, score) in enumerate(search(query), 1):
    print(f"{rank}. {text}  (similarity={score:.4f})")

100%|██████████| 1/1 [00:00<00:00,  2.30it/s]


1. RAG 是 Retrieval‑Augmented Generation 的缩写。  (similarity=1.0000)
2. Dual‑Encoder 采用查询和文档两个塔独立编码。  (similarity=0.4616)
3. 在图谱检索中常用图数据库 Neo4j。  (similarity=0.4595)


## Multi‑vector 检索
### 为每一个文档块创建子分块， 基于子分块和原始文档块进行匹配， 返回原始文档块。
### 为每一个文档块创建摘要(Summary)， 基于摘要和原始文档块进行匹配， 返回原始文档块。
### 为每一个文档块创建假设查询(Hypothetical Questions)， 基于假设查询和原始文档块进行匹配， 返回原始文档块。

In [18]:
# 引入uuid库，用于生成唯一标识符
import uuid
from langchain.retrievers import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_community.document_loaders.text import TextLoader
from langchain_community.vectorstores.chroma import Chroma
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import DashScopeEmbeddings
loaders = [TextLoader('曲面打印机说明书.txt', encoding='utf-8') ,TextLoader('text.txt', encoding='utf-8') ]
embeddings = DashScopeEmbeddings(
    model="text-embedding-v2",
    dashscope_api_key="sk-0b8d48b85f1742829ef3032133375d3e")
llm = ChatOpenAI(base_url='https://dashscope.aliyuncs.com/compatible-mode/v1',
                 api_key="sk-0b8d48b85f1742829ef3032133375d3e",
                 model="qwen2.5-72b-instruct", temperature=0.7)
# 创建一个RecursiveCharacterTextSplitter对象，用于分割文本
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000)
# 使用TextLoader对象加载文本文件
# docs = loader.load()
docs = []
for loader in loaders:
    docs.extend(loader.load())
# 使用RecursiveCharacterTextSplitter对象分割文档
docs = text_splitter.split_documents(docs)


# 定义一个字典，键为"doc"，值为一个函数，该函数返回文档的内容
chain = (
    {"doc": lambda x: x.page_content}
    # 使用ChatPromptTemplate类的from_template方法创建一个聊天提示模板
    | ChatPromptTemplate.from_template("总结以下文档:\n\n{doc}")
    # 创建一个ChatOpenAI对象，最大重试次数为0
    |llm
    # 创建一个StrOutputParser对象
    | StrOutputParser()
)
# 使用chain对象的batch方法批量处理文档，最大并发数为5
summaries = chain.batch(docs, {"max_concurrency": 5})

# 创建一个Chroma对象，集合名为"summaries"，嵌入函数为OpenAIEmbeddings()
vectorstore = Chroma(collection_name="summaries", embedding_function=embeddings)
# 创建一个InMemoryByteStore对象，用于存储字节数据
store = InMemoryByteStore()
# 定义一个字符串，值为"doc_id"
id_key = "doc_id"
# 创建一个MultiVectorRetriever对象
retriever = MultiVectorRetriever(
    # 设置向量存储为vectorstore对象
    vectorstore=vectorstore,
    # 设置字节存储为store对象
    byte_store=store,
    # 设置id键为"id_key"
    id_key=id_key,
)
# 为每个文档生成一个唯一的ID
doc_ids = [str(uuid.uuid4()) for _ in docs]
# 创建一个Document对象，页面内容为s，元数据为{id_key: doc_ids[i]}
summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    # 对summaries中的每一个元素和它的索引进行迭代
    for i, s in enumerate(summaries)
]
# 使用retriever对象的vectorstore属性的add_documents方法添加文档
retriever.vectorstore.add_documents(summary_docs)
# 使用retriever对象的docstore属性的mset方法设置文档
retriever.docstore.mset(list(zip(doc_ids, docs)))
# 使用vectorstore对象的similarity_search方法搜索与"如何写文章"相似的文档
sub_docs = vectorstore.similarity_search("张三")
# 获取搜索结果的第一个文档
print(sub_docs[0] )
# 使用retriever对象的get_relevant_documents方法获取与"如何写文章"相关的文档
retrieved_docs = retriever.get_relevant_documents("张三")
# 获取搜索结果的第一个文档的内容
print(retrieved_docs[0].page_content  )


## RGA QUERY改写
### 通过对原始查询进行语义扩展或改写后，提升召回效果，适用于 Dense 检索、多向量检索、RAG 等场景

In [18]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import os

# ===================== 配置区域 =====================
embeddings = DashScopeEmbeddings(
    model="text-embedding-v2",
    dashscope_api_key="sk-0b8d48b85f1742829ef3032133375d3e")
llm = ChatOpenAI(base_url='https://dashscope.aliyuncs.com/compatible-mode/v1',
                 api_key="sk-0b8d48b85f1742829ef3032133375d3e",
                 model="qwen2.5-72b-instruct", temperature=0.7)
client_openai = OpenAI(
    api_key= "sk-07b0bbfd24bf4cb391cad5da8da05f6f",
    base_url= "https://dashscope.aliyuncs.com/compatible-mode/v1" 
)
EXPANSION_NUM = 3   # RGA生成几个改写查询
TOP_K = 4           # 每个子查询召回K个文档

# ===================== 加载文档并构建索引 =====================
def build_faiss_index(filepath):
    loader = TextLoader(filepath, encoding="utf-8")
    docs = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    chunks = splitter.split_documents(docs)

   
    vectorstore = FAISS.from_documents(chunks, embeddings)
    return vectorstore

# ===================== 构建RGA查询扩充链 =====================
def build_query_expansion_chain():
   # llm = ChatOpenAI(model=client_openai, temperature=0.7)
    prompt = PromptTemplate.from_template("""
你是一个检索增强助手，请基于用户的原始查询，生成 {num} 个不同角度但相关的检索查询，增强语义多样性。

原始查询：{query}

请输出一组改写查询：
""")
    return LLMChain(llm=llm, prompt=prompt)

# ===================== 多查询检索并合并结果 =====================
def rga_retrieve(query, vectorstore, expander, top_k=4):
    # 1. RGA生成改写查询
    expansion_prompt = expander.run({"query": query, "num": EXPANSION_NUM})
    reformulated_queries = [line.strip(" 123456.-").strip() 
                            for line in expansion_prompt.strip().split("\n") if line.strip()]
    
    print(f"\n🧠 原始查询: {query}")
    print("🔁 改写查询:")
    for i, rq in enumerate(reformulated_queries):
        print(f"  [{i+1}] {rq}")

    # 2. 每个子查询分别检索
    all_results = []
    for rq in reformulated_queries:
        results = vectorstore.similarity_search(rq, k=top_k)
        all_results.extend(results)

    # 3. 去重（可选：再排序）
    seen = set()
    unique_results = []
    for d in all_results:
        if d.page_content not in seen:
            seen.add(d.page_content)
            unique_results.append(d)

    return unique_results[:top_k * EXPANSION_NUM]

# ===================== 主流程 =====================
if __name__ == "__main__":
    vs = build_faiss_index("曲面打印机说明书.txt")
    expander = build_query_expansion_chain()

    while True:
        query = input("\n请输入查询（空退出）：").strip()
        if not query:
            break
        results = rga_retrieve(query, vs, expander)

        print(f"\n📚 共返回 {len(results)} 条结果：")
        for i, doc in enumerate(results):
            print(f"\n[{i+1}] {doc.metadata.get('source', '')}")
            print(doc.page_content[:200] + "…")


## RGA 层级稠密检索代码

In [25]:
import os, faiss, numpy as np
from typing import Dict
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# ====== 配置 ======
DASH_API_KEY = "sk-0b8d48b85f1742829ef3032133375d3e"
EMBED_MODEL = "text-embedding-v2"
LLM_MODEL = "qwen2.5-72b-instruct"
EMBED_DIM = 256
DOC_EMB   = DashScopeEmbeddings(model=EMBED_MODEL, dashscope_api_key=DASH_API_KEY)
CHUNK_EMB = DOC_EMB
LLM       =  ChatOpenAI(base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
                 api_key=DASH_API_KEY,
                 model=LLM_MODEL, temperature=0.7)
L1, L2 = 3, 4                     # doc / chunk top‑k
EXP_N   = 3                       # RGA 改写数


SPLITTER= RecursiveCharacterTextSplitter(chunk_size=120, chunk_overlap=20)

# ====== 1. 自带微型语料 ======
RAW = [
    "打印机应在通风环境下使用，防止设备过热并降低火灾风险。",
    "维修打印机前必须断电，以避免触电和机械伤害。",
    "节能模式可让打印机功耗降低 30%。"
]
docs   = [Document(page_content=t, metadata={"id": f"d{i}"}) for i, t in enumerate(RAW)]
chunks = SPLITTER.split_documents(docs)

# ====== 2. 建双索引 ======
doc_vec = np.array(DOC_EMB.embed_documents([d.page_content for d in docs]), dtype="float32")
chk_vec = np.array(CHUNK_EMB.embed_documents([c.page_content for c in chunks]), dtype="float32")
for v in (doc_vec, chk_vec): faiss.normalize_L2(v)
dim = doc_vec.shape[1]          # ← 自动抓取真实维度（应为 1536）
doc_idx = faiss.IndexFlatIP(dim)
doc_idx.add(doc_vec)

chk_idx = faiss.IndexFlatIP(dim)
chk_idx.add(chk_vec)

chk2doc = [c.metadata["id"] for c in chunks]
id2doc  = {d.metadata["id"]: d for d in docs}

# ====== 3. RGA 扩写器 ======
expander = LLMChain(
    llm=LLM,
    prompt=PromptTemplate.from_template(
        "给用户查询扩写 {n} 个不同表达，每行一条：\n查询：{q}\n扩写："
    )
)

# ====== 4. 层级检索函数 ======
def search(q: str):
    exps = expander.run({"q": q, "n": EXP_N})
    queries = [q] + [l.strip(" 123456.-") for l in exps.split("\n") if l.strip()]
    print("🔍 扩展查询:", queries)

    # 文档级
    qv = np.array([DOC_EMB.embed_query(x) for x in queries], dtype="float32"); faiss.normalize_L2(qv)
    _, I = doc_idx.search(qv, L1)
    cand_ids = {docs[i].metadata["id"] for i in I.flatten()}

    # chunk 级
    qv2 = np.array([CHUNK_EMB.embed_query(x) for x in queries], dtype="float32"); faiss.normalize_L2(qv2)
    D, I = chk_idx.search(qv2, L2)

    scores: Dict[str, float] = {}
    for ds, is_ in zip(D, I):
        for s, idx in zip(ds, is_):
            did = chk2doc[idx]
            if did in cand_ids:
                scores[did] = max(scores.get(did, -1), s)

    return sorted([(id2doc[k], v) for k, v in scores.items()], key=lambda x: x[1], reverse=True)

# ====== 5. Demo 运行 ======
for doc, sc in search("打印机安全隐患")[:3]:
    print(f"\nScore={sc:.3f} | {doc.page_content}")


APIConnectionError: Connection error.

## RGA + FiD 风格 检索 

In [18]:
import os, faiss, numpy as np
from typing import Dict
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# ====== 配置 ======
DASH_API_KEY = "sk-0b8d48b85f1742829ef3032133375d3e"
EMBED_MODEL = "text-embedding-v2"
LLM_MODEL = "qwen2.5-72b-instruct"
EMBED_DIM = 256
EMBEDDING   = DashScopeEmbeddings(model=EMBED_MODEL, dashscope_api_key=DASH_API_KEY)

LLM       =  ChatOpenAI(base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
                 api_key=DASH_API_KEY,
                 model=LLM_MODEL, temperature=0.7)


CHUNK_SIZE = 150
CHUNK_OVERLAP = 20
TOP_K = 4
EXPANSION_N = 3

# ===== 微型语料库 =====
RAW_TEXTS = [
    "打印机应放置在通风良好的环境中以避免过热。",
    "使用打印机前应检查电缆连接是否牢固。",
    "维护打印机前，请确保设备断电以避免触电风险。",
    "节能模式可以将打印机能耗降低最多30%。",
    "使用原装耗材可以提高打印质量和设备寿命。"
]
docs = [Document(page_content=t, metadata={"id": str(i)}) for i, t in enumerate(RAW_TEXTS)]

# ===== 切分并构建 chunk 索引 =====
splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
chunks = splitter.split_documents(docs)
chunk_vecs = np.array(EMBEDDING.embed_documents([c.page_content for c in chunks]), dtype="float32")
faiss.normalize_L2(chunk_vecs)
index = faiss.IndexFlatIP(chunk_vecs.shape[1])
index.add(chunk_vecs)

# mapping
chunk2doc = chunks
doc_id_map = {i: c for i, c in enumerate(chunks)}

# ===== 查询改写器（RGA） =====
query_expander = LLMChain(
    llm=LLM,
    prompt=PromptTemplate.from_template(
        "你是一名查询增强助手。请将以下查询改写为 {n} 条不同措辞的表达，每行一条：\n用户查询：{query}\n改写："
    )
)

# ===== FiD 风格问答 Prompt =====
fid_prompt = PromptTemplate.from_template(
    "根据以下多个参考内容，回答用户问题。\n\n参考信息：\n{context}\n\n用户问题：{question}\n\n回答："
)

# ===== 检索增强回答流程 =====
def rga_fid_qa(query: str):
    # (1) 扩写查询
    expanded = query_expander.run({"query": query, "n": EXPANSION_N})
    queries = [query] + [q.strip(" 123456.-") for q in expanded.split("\n") if q.strip()]
    print("🔍 扩写查询：", queries)

    # (2) 分别检索 top-k chunk
    all_chunks = []
    for q in queries:
        q_vec = np.array(EMBEDDING.embed_query(q), dtype="float32").reshape(1, -1)
        faiss.normalize_L2(q_vec)
        _, I = index.search(q_vec, TOP_K)
        all_chunks.extend([doc_id_map[i] for i in I[0]])

    # 去重
    unique_chunks = {c.page_content for c in all_chunks}
    context = "\n".join(unique_chunks)

    # (3) 拼接输入 -> FiD 风格生成
    final_prompt = fid_prompt.format(context=context, question=query)
    answer = LLM.invoke(final_prompt).content
    return answer

# ===== 运行示例 =====
if __name__ == "__main__":
    q = "如何保障打印机使用过程的安全？"
    result = rga_fid_qa(q)
    print("\n🧠 回答：", result)


## RGA 压缩稠密检索的定义

In [ ]:
import os
import numpy as np
from typing import List
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS

# ====== 配置 ======
DASH_API_KEY = "sk-0b8d48b85f1742829ef3032133375d3e"
EMBED_MODEL = "text-embedding-v2"
LLM_MODEL = "qwen2.5-72b-instruct"

CHUNK_SIZE = 300
CHUNK_OVERLAP = 50
EMBED_DIM = 256  # ✅ 压缩维度
TOP_K = 5
EXPANSION_N = 3

# ====== 向量模型 ======
embedding =  DashScopeEmbeddings(model=EMBED_MODEL, dashscope_api_key=DASH_API_KEY)
llm = ChatOpenAI(base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
                 api_key=DASH_API_KEY,
                 model=LLM_MODEL, temperature=0.7)

# ====== ① 加载数据并构建索引 ======
def load_and_index(doc_path: str):
    loader = TextLoader(doc_path, encoding="utf-8")
    docs = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    chunks = splitter.split_documents(docs)
    vectorstore = FAISS.from_documents(chunks, embedding)
    return vectorstore

# ====== ② 构建查询扩展器（RGA）======
def build_query_expander():
    prompt = PromptTemplate.from_template(
        """你是一名检索增强助手，请将用户输入的查询，生成 {n} 条不同关键词或表达方式的扩展查询，用换行分隔：
        用户查询：{query}
        扩展查询："""
    )
    return LLMChain(llm=llm, prompt=prompt)

# ====== ③ 压缩稠密检索 + 查询扩展 ======
def rga_compressed_retrieve(query: str, vectorstore: FAISS, expander: LLMChain):
    # (1) 扩展查询
    expanded = expander.run({"query": query, "n": EXPANSION_N})
    queries = [query] + [q.strip(" 123456.-") for q in expanded.split("\n") if q.strip()]
    print("🔍 扩展查询：", queries)

    # (2) 多查询检索
    seen = set()
    results = []
    for q in queries:
        docs = vectorstore.similarity_search(q, k=TOP_K)
        for d in docs:
            if d.page_content not in seen:
                results.append(d)
                seen.add(d.page_content)
    return results[:TOP_K]

# ====== ④ 主函数 ======
if __name__ == "__main__":
    vs = load_and_index("曲面打印机说明书.txt")  # 替换为你的文档
    expander = build_query_expander()
    query = "打印机的节能设计有哪些？"

    results = rga_compressed_retrieve(query, vs, expander)
    print(f"\n🎯 Top {len(results)} 段落：")
    for i, d in enumerate(results, 1):
        print(f"\n[{i}] {d.page_content[:200]}...")


## Hybrid Dense + Metadata 检索
###   特点：
RGA（Retrieval with Generated Augmentation）：查询改写增强

Dense 向量检索：基于 OpenAI 或 DashScope embedding

Metadata（元数据）过滤：基于文档来源、时间、作者等条件过滤文档

Hybrid 检索：稠密 + 结构信息（metadata）双通道筛选

In [ ]:
import os
from typing import List
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.docstore.document import Document

# ====== 配置 ======
DASH_API_KEY = "sk-0b8d48b85f1742829ef3032133375d3e"
EMBED_MODEL = "text-embedding-v2"
LLM_MODEL = "qwen2.5-72b-instruct"

CHUNK_SIZE = 300
CHUNK_OVERLAP = 50
EMBED_DIM = 256  # ✅ 压缩维度
TOP_K = 5
EXPANSION_N = 3

# ====== 向量模型 ======
embedding =  DashScopeEmbeddings(model=EMBED_MODEL, dashscope_api_key=DASH_API_KEY)
llm = ChatOpenAI(base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
                 api_key=DASH_API_KEY,
                 model=LLM_MODEL, temperature=0.7)

# ========== ① 加载文档 ==========
def load_documents(path: str, metadata_value: str):
    loader = TextLoader(path, encoding="utf-8")
    docs = loader.load()

    # 加入元信息 metadata
    for doc in docs:
        doc.metadata["category"] = metadata_value  # 可扩展多个字段
    return docs

# ========== ② 建索引 ==========
def build_index(docs: List[Document]):
    splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    chunks = splitter.split_documents(docs)
    return FAISS.from_documents(chunks, embedding)

# ========== ③ 查询扩写器 ==========
def build_expander():
    prompt = PromptTemplate.from_template(
        """你是一名检索助手，扩写下列用户查询为 {n} 条不同表达的检索式，用换行分隔：
        用户查询：{query}
        扩写："""
    )
    return LLMChain(llm=llm, prompt=prompt)

# ========== ④ 混合检索函数 ==========
def rga_hybrid_search(query: str, index: FAISS, expander: LLMChain,
                      metadata_filter: dict = None):
    # 1. 查询扩写
    expansion = expander.run({"query": query, "n": EXPANSION_N})
    queries = [query] + [q.strip(" 123456.-") for q in expansion.split("\n") if q.strip()]
    print("🔍 扩展查询：", queries)

    # 2. 查询 + 元信息过滤
    matched_docs = []
    seen = set()

    for q in queries:
        docs = index.similarity_search(q, k=TOP_K)
        for doc in docs:
            # ✅ metadata 过滤
            if metadata_filter:
                passed = all(doc.metadata.get(k) == v for k, v in metadata_filter.items())
                if not passed:
                    continue
            if doc.page_content not in seen:
                matched_docs.append(doc)
                seen.add(doc.page_content)

    return matched_docs[:TOP_K]

# ========== ⑤ 主程序 ==========
if __name__ == "__main__":
    # 模拟加载多种类型文档
    safety_docs = load_documents("曲面打印机说明书.txt", metadata_value="safety")
    usage_docs = load_documents("text.txt", metadata_value="usage")

    all_docs = safety_docs + usage_docs
    index = build_index(all_docs)
    expander = build_expander()

    user_query = "打印机可能有哪些危险因素？"
    filtered_metadata = {"category": "safety"}  # ✅ 指定过滤条件

    results = rga_hybrid_search(user_query, index, expander, filtered_metadata)

    print(f"\n🎯 Top {len(results)} 结果：")
    for i, doc in enumerate(results, 1):
        print(f"\n[{i}] {doc.metadata} \n{doc.page_content[:200]}...")


## RAG-Fusion

In [ ]:
import os
import openai
import random

# Initialize OpenAI API
openai.api_key = os.getenv("OPENAI_API_KEY")  # Alternative: Use environment variable
if openai.api_key is None:
    raise Exception("No OpenAI API key found. Please set it as an environment variable or in main.py")

# Function to generate queries using OpenAI's ChatGPT
def generate_queries_chatgpt(original_query):

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates multiple search queries based on a single input query."},
            {"role": "user", "content": f"Generate multiple search queries related to: {original_query}"},
            {"role": "user", "content": "OUTPUT (4 queries):"}
        ]
    )

    generated_queries = response.choices[0]["message"]["content"].strip().split("\n")
    return generated_queries

# Mock function to simulate vector search, returning random scores
def vector_search(query, all_documents):
    available_docs = list(all_documents.keys())
    random.shuffle(available_docs)
    selected_docs = available_docs[:random.randint(2, 5)]
    scores = {doc: round(random.uniform(0.7, 0.9), 2) for doc in selected_docs}
    return {doc: score for doc, score in sorted(scores.items(), key=lambda x: x[1], reverse=True)}

# Reciprocal Rank Fusion algorithm
def reciprocal_rank_fusion(search_results_dict, k=60):
    fused_scores = {}
    print("Initial individual search result ranks:")
    for query, doc_scores in search_results_dict.items():
        print(f"For query '{query}': {doc_scores}")
        
    for query, doc_scores in search_results_dict.items():
        for rank, (doc, score) in enumerate(sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)):
            if doc not in fused_scores:
                fused_scores[doc] = 0
            previous_score = fused_scores[doc]
            fused_scores[doc] += 1 / (rank + k)
            print(f"Updating score for {doc} from {previous_score} to {fused_scores[doc]} based on rank {rank} in query '{query}'")

    reranked_results = {doc: score for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)}
    print("Final reranked results:", reranked_results)
    return reranked_results

# Dummy function to simulate generative output
def generate_output(reranked_results, queries):
    return f"Final output based on {queries} and reranked documents: {list(reranked_results.keys())}"


# Predefined set of documents (usually these would be from your search database)
all_documents = {
    "doc1": "Climate change and economic impact.",
    "doc2": "Public health concerns due to climate change.",
    "doc3": "Climate change: A social perspective.",
    "doc4": "Technological solutions to climate change.",
    "doc5": "Policy changes needed to combat climate change.",
    "doc6": "Climate change and its impact on biodiversity.",
    "doc7": "Climate change: The science and models.",
    "doc8": "Global warming: A subset of climate change.",
    "doc9": "How climate change affects daily weather.",
    "doc10": "The history of climate change activism."
}

# Main function
if __name__ == "__main__":
    original_query = "impact of climate change"
    generated_queries = generate_queries_chatgpt(original_query)
    
    all_results = {}
    for query in generated_queries:
        search_results = vector_search(query, all_documents)
        all_results[query] = search_results
    
    reranked_results = reciprocal_rank_fusion(all_results)
    
    final_output = generate_output(reranked_results, generated_queries)
    
    print(final_output)